In [ ]:
3 am commit try

In [1]:
import pandas as pd
import sys
from tkinter import Tk, filedialog
import ast
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from tabulate import tabulate
import pprint 
import json
import scipy.stats as stats
import json
import pickle
import ipywidgets as widgets
from IPython.display import display
from nbconvert.preprocessors import ExecutePreprocessor
from nbformat import read, write
import nbformat
from functools import reduce

## Upload Database and Filter

In [11]:
# Load the DataFrame from the pkl file
file_path = '/Users/kaycei/K on Server (NoMachine)/KayCei/Naturalistic.pkl'
with open(file_path, 'rb') as f:
    df = pickle.load(f)

# Identify columns with 'Array' in their names and parse JSON strings in those columns into Python objects
df['Selected Behavior'] = df['Selected Behavior'].str.strip()
columns_to_parse = [col for col in df.columns if 'Array' in col]
for col in columns_to_parse:
    df[col] = df[col].apply(json.loads)


# Define a function to convert nested lists to nested tuples
def convert_to_tuples(lst):
    if isinstance(lst, list):
        return tuple(convert_to_tuples(item) for item in lst)
    else:
        return lst

# Convert lists of lists into lists of tuples
for col in columns_to_parse:
    df[col] = df[col].apply(convert_to_tuples)

# Check if conversion was successful and print a statement accordingly
if all(isinstance(item, list) and all(isinstance(subitem, tuple) for subitem in item) for col in columns_to_parse for item in df[col]):
    print("JSON strings in columns with 'Array' in their names have been successfully converted to lists of tuples.")
else:
    print("Conversion of JSON strings to lists of tuples in columns with 'Array' in their names was unsuccessful.")

Conversion of JSON strings to lists of tuples in columns with 'Array' in their names was unsuccessful.


In [12]:
# Replace values in 'Selected_Behavior' column where 'Session_Type' is 'Sham Session'
df.loc[df['Session Type'] == 'Sham Session', 'Selected Behavior'] = 'No Error Value'
df.loc[df['Session Type'] == 'Sham Session', 'Fiber Connection'] = 'No Error Value'

In [13]:
original_df=df.copy()

In [14]:
iteration_list = {}

# Display available columns in groups of 3 with aligned numbering
num_columns = 3
columns = df.columns.tolist()
num_options = len(columns)

# Calculate the width of each column
column_width = 32

# Define the number of spaces for tab spacing
tab_space = 1

# Set the number of rows for displaying column options
num_rows = 13

# Initialize iteration number
iteration = 0
iteration_list[iteration]=("Selected Header: None","Column:None",df)

while columns:  # Continue filtering until all headers are filtered
    filtering_criteria = []
    for i in range(num_rows):
        for j in range(i, len(columns), num_rows):
            col_num = j + 1
            col_name = columns[j]
            # Adjust font size for smaller font
            if len(col_name) > column_width:
                print(f"\033[1;33m{col_num:<2}.\033[1;32m{col_name[:column_width]:<{column_width}}\033[0m", end=" " * tab_space)  # Yellow color for smaller font size
            else:
                print(f"\033[1;33m{col_num:<2}.{col_name:<{column_width}}\033[0m", end=" " * tab_space)
        print()
        
    if len(iteration_list)!=1:
        df=current_match
    else:
        pass
    
    # Prompt user to enter the number corresponding to the header
    selected_option = None
    while selected_option is None:
        user_input = input("Enter the number corresponding to the header or '0' to cancel filtering: ")
        try:
            selected_option = int(user_input)
            if selected_option == 0:
                print("Cell execution stopped.")
                sys.exit()  # Exit just the current cell
            elif selected_option > len(columns):
                raise ValueError
        except ValueError:
            print(f"Invalid input. Please enter a number between 1 and {len(columns)} or '0' to cancel filtering.")
            selected_option = None

    print((columns[selected_option-1]))
    selected_header = columns[selected_option - 1]
    
    # Append the selected header to filtering_criteria
    filtering_criteria.append(f"Selected Header: {selected_header}")

    # Remove the selected header from the list of available columns
        
    if selected_header in df.columns and all(isinstance(value, str) for value in df[selected_header]):
        avaliable_value=df[selected_header].unique()
        print(avaliable_value)
        filter_value = input(f"Enter the value to filter by {selected_header}: ")
        # Append the filter value to filtering_criteria
        filtering_criteria.append(f"Filter Value: {filter_value}")

    elif 'Array' in selected_header:
        available_elements = {
            0:"Behavior",
            1:"Stim Count",
            2: "Frames", 
            3: "Bout", 
            4: "Duration",
            5: "Start Time",
            6: "End Time",
        }
        print("Available elements:")
        for key, value in available_elements.items():
            print(f"{key}: {value}")

        # Prompt user to select elements
        element_positions_input = input(f"Enter the elements you want to filter by {selected_header} (comma-separated): ")
        if ',' in element_positions_input:
            selected_positions = [int(pos.strip()) for pos in element_positions_input.split(',')]
            # Check if input is valid
            if all(pos in available_elements for pos in selected_positions):
                combined_filters = []
                for pos in selected_positions:
                    range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[pos]}?: ")
                    if range_or_exact.lower() == 'exact':
                        filter_value = input(f"Enter the exact value for {available_elements[pos]}: ")
                        # Add condition to the combined filter
                        combined_filters.append(df[selected_header].apply(lambda row: any((x[pos] == filter_value for x in tup)for tup in row)))
                        filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                        filtering_criteria.append(f"Exact Value for {available_elements[pos]}: {filter_value}")
                    elif range_or_exact.lower() == 'range':
                        range_value = input(f"Enter the range for {available_elements[pos]} (min,max): ")
                        min_val, max_val = map(float, range_value.split(','))
                        # Add condition to the combined filter
                        combined_filters.append(df[selected_header].apply(lambda row: any((min_val <= x[pos]<= max_val for x in tup)for tup in row)))
                        filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                        filtering_criteria.append(f"Range for {available_elements[pos]}: {min_val} to {max_val}")
                    else:
                        print("Invalid input. Please enter 'exact' or 'range'.")
                # Apply combined filter
                array_df = df[reduce(lambda x, y: x & y, combined_filters)]
            else:
                print("Value is not an elemental position")
        else:
            selected_positions = int(element_positions_input)
            print(selected_positions)
            if selected_positions in available_elements:
                range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[selected_positions]}?: ")
                if range_or_exact.lower() == 'exact':
                    filter_value= input(f"Enter the exact value for {available_elements[selected_positions]}: ")
                    filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                    filtering_criteria.append(f"Exact Value for {available_elements.get(selected_positions)}: {filter_value}")
                    array_df = df[df[selected_header].apply(lambda row: any((x[selected_positions] == filter_value for x in tup) for tup in row))]
                elif range_or_exact== 'range':
                    range_value = input(f"Enter the range for {available_elements[selected_positions]} (min,max): ")
                    min_val, max_val = map(float, range_value.split(','))
                    filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                    filtering_criteria.append(f"Range for {available_elements.get(pos)}: {min_val} to {max_val}")
                    array_df=df[df[selected_header].apply(lambda y: any((min_val <= x[selected_positions] <= max_val for x in tup)for tup in y))]
            else:
                print('An element position was not selected')

    if 'Array'not in selected_header:
        columns.remove(selected_header)
    
    if len(iteration_list)==1:
        if "Array" not in selected_header:                                          
            current_match=df[df[selected_header] == filter_value]
            iteration+=1
            iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)
        elif "Selected Behavior" or 'Fiber Connection' in selected_header:
            current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
            iteration += 1
            iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
        else:
            current_match=array_df
            iteration+=1
            iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       
    else:
        previous_match=current_match
        if'Array' in selected_header:
            current_match=array_df
            iteration+=1
            iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match) 
        elif "Selected Behavior" or 'Fiber Connection' in selected_header:
            current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
            iteration += 1
            iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
        else:
            current_match=df[df[selected_header]==filter_value]
            iteration+=1
            iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                        
    
    # Extracting the headers in the original order from the DataFrame
    original_headers = current_match.columns.tolist()



1 .Mouse                            14.Total Left Turn Bout Count       27.Face Groom Average Duration in S 
2 .Genotype                         15.Average Duration of Left Turn(se 28.All Other Groom Array            
3 .Date                             16.Total Left Turn Bouts in Stim Bl 29.Total All Other Groom Bout Count 
4 .Selected Behavior                17.Left Turn Average Duration in St 30.Average Duration of All Other Gr 
5 .Session Type                     18.Locomotion Array                 31.Total All Other Groom Bouts in S 
6 .Session Number                   19.Total Locomotion Bout Count      32.All Other Groom Average Duration 
7 .Fiber Connection                 20.Average Duration of Locomotion(s 33.All Other Array                  
8 .Right Turn Array                 21.Total Locomotion Bouts in Stim B 34.Total All Other Bout Count       
9 .Total Right Turn Bout Count      22.Locomotion Average Duration in S 35.Average Duration of All Other(se 
10.Average Duration

SystemExit: 

/Users/kaycei/anaconda3/envs/closed-loop/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [123]:
for iteration, (filtering_criteria, columns, current_match) in iteration_list.items():
    print(iteration, filtering_criteria)

0 Selected Header: None
1 ['Selected Header: Genotype', 'Filter Value: D1']
2 ['Selected Header: Selected Behavior', 'Filter Value: Right Turn']
4 ['Selected Header: Fiber Connection', 'Filter Value: Ipsilateral']


iteration_list.clear()
filtering_criteria.clear()

In [111]:
len(anovadf[anovadf['Session Type'] == 'Stim Session'])

4

In [195]:
while True:
    finalize_match = input("\nDo you want to finalize matching data? (yes/no): ")
    if finalize_match.lower() == 'yes':
        break  # Exit the loop if the user chooses to finalize matching data
    
    for iteration, (filtering_criteria, columns, current_match) in iteration_list.items():
        print(iteration, filtering_criteria)

    # Prompt the user to select an iteration to go back to
    selected_iteration = input("\nEnter the iteration number to go back to, or 'exit' to exit: ")
    if selected_iteration.lower() == 'exit':
        break  # Exit the loop if the user chooses to exit
    else:
        try:
            selected_iteration = int(selected_iteration)
            if selected_iteration in iteration_list:
                df=iteration_list[selected_iteration][2]
                current_match = iteration_list[selected_iteration][2]
                iteration_list = {k: v for k, v in iteration_list.items() if k <= selected_iteration}
                print('Run cell above as current match')
                break
        except ValueError:
            print("Invalid input. Please enter a valid iteration number or 'exit' to exit.")




Do you want to finalize matching data? (yes/no): no
0 Selected Header: None
1 ['Selected Header: Genotype', 'Filter Value: D1']
6 ['Selected Header: Selected Behavior', 'Filter Value: Locomotion']

Enter the iteration number to go back to, or 'exit' to exit: 6
Run cell above as current match


In [196]:
if finalize_match!='yes':
    # Define a list to store all selections and filtering criteria
    #iteration_list={}
    # Display available columns in groups of 3 with aligned numbering
    num_columns = 3
    columns = df.columns.tolist()
    num_options = len(columns)

    # Calculate the width of each column
    column_width = 32

    # Define the number of spaces for tab spacing
    tab_space = 1

    # Set the number of rows for displaying column options
    num_rows = 13

    while columns:  # Continue filtering until all headers are filtered
        for i in range(num_rows):
            for j in range(i, len(columns), num_rows):
                col_num = j + 1
                col_name = columns[j]
                # Adjust font size for smaller font
                if len(col_name) > column_width:
                    print(f"\033[1;33m{col_num:<2}.\033[1;32m{col_name[:column_width]:<{column_width}}\033[0m", end=" " * tab_space)  # Yellow color for smaller font size
                else:
                    print(f"\033[1;33m{col_num:<2}.{col_name:<{column_width}}\033[0m", end=" " * tab_space)
            print()

        if len(iteration_list)!=1:
            df=current_match
        else:
            pass
        filtering_criteria = []
        # Prompt user to enter the number corresponding to the header
        selected_option = None
        while selected_option is None:
            user_input = input("Enter the number corresponding to the header or '0' to cancel filtering: ")
            try:
                selected_option = int(user_input)
                if selected_option == 0:
                    print("Cell execution stopped.")
                    sys.exit()  # Exit just the current cell
                elif selected_option > len(columns):
                    raise ValueError
            except ValueError:
                print(f"Invalid input. Please enter a number between 1 and {len(columns)} or '0' to cancel filtering.")
                selected_option = None

        print((columns[selected_option-1]))
        selected_header = columns[selected_option - 1]

        # Append the selected header to filtering_criteria
        filtering_criteria.append(f"Selected Header: {selected_header}")

        # Remove the selected header from the list of available columns

        if selected_header in df.columns and all(isinstance(value, str) for value in df[selected_header]):
            avaliable_value=df[selected_header].unique()
            print(avaliable_value)
            filter_value = input(f"Enter the value to filter by {selected_header}: ")
            # Append the filter value to filtering_criteria
            filtering_criteria.append(f"Filter Value: {filter_value}")

        elif 'Array' in selected_header:
            available_elements = {
                0:"Behavior",
                1:"Stim Count",
                2: "Frames", 
                3: "Bout", 
                4: "Duration",
                5: "Start Time",
                6: "End Time",
            }
            print("Available elements:")
            for key, value in available_elements.items():
                print(f"{key}: {value}")

            # Prompt user to select elements
            element_positions_input = input(f"Enter the elements you want to filter by {selected_header} (comma-separated): ")
            print("Selected positions:", selected_positions)  # Add this line for debugging
            if ',' in element_positions_input:
                selected_positions = [int(pos.strip()) for pos in element_positions_input.split(',')]
                # Check if input is valid
                if all(pos in available_elements for pos in selected_positions):
                    combined_filters = []
                    for pos in selected_positions:
                        range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[pos]}?: ")
                        if range_or_exact.lower() == 'exact':
                            filter_value = input(f"Enter the exact value for {available_elements[pos]}: ")
                            # Add condition to the combined filter
                            combined_filters.append(df[selected_header].apply(lambda row: any((x[pos] == filter_value for x in tup)for tup in row)))
                            filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                            filtering_criteria.append(f"Exact Value for {available_elements[pos]}: {filter_value}")
                        elif range_or_exact.lower() == 'range':
                            range_value = input(f"Enter the range for {available_elements[pos]} (min,max): ")
                            min_val, max_val = map(float, range_value.split(','))
                            # Add condition to the combined filter
                            combined_filters.append(df[selected_header].apply(lambda row: any((min_val <= x[pos]<= max_val for x in tup)for tup in row)))
                            filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                            filtering_criteria.append(f"Range for {available_elements[pos]}: {min_val} to {max_val}")
                        else:
                            print("Invalid input. Please enter 'exact' or 'range'.")
                    # Apply combined filter
                    array_df = df[reduce(lambda x, y: x & y, combined_filters)]
                else:
                    print("Value is not an elemental position")
            else:
                selected_positions = int(element_positions_input)
                print(selected_positions)
                if selected_positions in available_elements:
                    range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[selected_positions]}?: ")
                    if range_or_exact.lower() == 'exact':
                        filter_value= input(f"Enter the exact value for {available_elements[selected_positions]}: ")
                        filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                        filtering_criteria.append(f"Exact Value for {available_elements.get(selected_positions)}: {filter_value}")
                        array_df = df[df[selected_header].apply(lambda row: any((x[selected_positions] == filter_value for x in tup) for tup in row))]
                    elif range_or_exact== 'range':
                        range_value = input(f"Enter the range for {available_elements[selected_positions]} (min,max): ")
                        min_val, max_val = map(float, range_value.split(','))
                        filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                        filtering_criteria.append(f"Range for {available_elements.get(pos)}: {min_val} to {max_val}")
                        array_df=df[df[selected_header].apply(lambda y: any((min_val <= x[selected_positions] <= max_val for x in tup)for tup in y))]
                else:
                    print('An element position was not selected')

        if 'Array'not in selected_header:
            columns.remove(selected_header)

        if len(iteration_list)==1:
            if "Array" not in selected_header:                                          
                current_match=df[df[selected_header] == filter_value]
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)
            elif "Selected Behavior" or 'Fiber Connection' in selected_header:
                current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
                iteration += 1
                iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
            else:
                current_match=array_df
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       
        else:
            previous_match=current_match
            if'Array' in selected_header:
                current_match=array_df
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match) 
            elif "Selected Behavior" or 'Fiber Connection' in selected_header:
                current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
                iteration += 1
                iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
            else:
                current_match=df[df[selected_header]==filter_value]
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       

        # Extracting the headers in the original order from the DataFrame
        original_headers = current_match.columns.tolist()
else:
    pass


1 .Mouse                            14.Total Left Turn Bout Count       27.Face Groom Average Duration in S 
2 .Genotype                         15.Average Duration of Left Turn(se 28.All Other Groom Array            
3 .Date                             16.Total Left Turn Bouts in Stim Bl 29.Total All Other Groom Bout Count 
4 .Selected Behavior                17.Left Turn Average Duration in St 30.Average Duration of All Other Gr 
5 .Session Type                     18.Locomotion Array                 31.Total All Other Groom Bouts in S 
6 .Session Number                   19.Total Locomotion Bout Count      32.All Other Groom Average Duration 
7 .Fiber Connection                 20.Average Duration of Locomotion(s 33.All Other Array                  
8 .Right Turn Array                 21.Total Locomotion Bouts in Stim B 34.Total All Other Bout Count       
9 .Total Right Turn Bout Count      22.Locomotion Average Duration in S 35.Average Duration of All Other(se 
10.Average Duration

SystemExit: 

/Users/kaycei/anaconda3/envs/closed-loop/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


for iteration, (filtering_criteria, columns, current_match) in iteration_list.items():
    print(iteration, filtering_criteria)

In [197]:
current_match

,Mouse,Genotype,Date,Selected Behavior,Session Type,Session Number,Fiber Connection,Right Turn Array,Total Right Turn Bout Count,Average Duration of Right Turn(seconds),...,All Other Groom Array,Total All Other Groom Bout Count,Average Duration of All Other Groom(seconds),Total All Other Groom Bouts in Stim Block,All Other Groom Average Duration in Stim Block(seconds),All Other Array,Total All Other Bout Count,Average Duration of All Other(seconds),Total All Other Bouts in Stim Block,All Other Average Duration in Stim Block(seconds)
1,AD12,D1,022024,Locomotion,Stim Session,1,Bilateral,"((1, 0, 3, 1, 1, 13, 14), (1, 1, 9, 2, 3, 19, ...",789,0.199,...,"((4, 0, 3, 1, 1, 2011, 2012), (4, 0, 3, 2, 1, ...",484,0.219,64,0.24,"((5, 1, 12, 1, 4, 0, 4), (5, 3, 24, 2, 8, 5, 1...",3405,0.771,1517,0.78
4,AD14,D1,022624,Locomotion,Stim Session,1,Bilateral,"((1, 1, 12, 1, 4, 46, 50), (1, 0, 3, 2, 1, 62,...",249,0.143,...,"((4, 0, 3, 1, 1, 97, 98), (4, 0, 3, 2, 1, 373,...",348,0.182,144,0.16,"((5, 0, 3, 1, 1, 0, 1), (5, 1, 6, 2, 2, 12, 14...",2487,1.303,1072,1.44
46,AD10,D1,062623,Locomotion,Stim Session,1,Bilateral,"((0, 1, 6, 1, 2, 81, 83), (0, 0, 3, 2, 1, 168,...",334,0.216,...,"((4, 0, 3, 1, 1, 380, 381), (4, 0, 3, 2, 1, 42...",82,0.23,26,0.13,"((5, 4, 36, 1, 12, 0, 12), (5, 1, 6, 2, 2, 18,...",1625,2.009,821,1.8
56,AD10,D1,072723,Locomotion,Stim Session,2,Bilateral,"((0, 1, 6, 1, 2, 4, 6), (0, 1, 6, 2, 2, 129, 1...",850,0.204,...,"((4, 1, 9, 1, 3, 309, 312), (4, 0, 3, 2, 1, 31...",81,0.16,41,0.16,"((5, 0, 3, 1, 1, 3, 4), (5, 5, 45, 2, 15, 6, 2...",1964,1.663,954,1.55
59,AD10,D1,102423,Locomotion,Stim Session,3,Bilateral,"((0, 3, 24, 1, 8, 54, 62), (0, 0, 3, 2, 1, 303...",420,0.257,...,"((4, 0, 3, 1, 1, 1128, 1129), (4, 0, 3, 2, 1, ...",78,0.549,18,0.15,"((5, 4, 39, 1, 13, 0, 13), (5, 1, 9, 2, 3, 14,...",1896,1.583,923,1.46
61,AD11,D1,051223,Locomotion,Stim Session,1,Bilateral,"((0, 0, 3, 1, 1, 52, 53), (0, 2, 18, 2, 6, 287...",890,0.206,...,"((4, 1, 6, 1, 2, 145, 147), (4, 0, 3, 2, 1, 28...",318,0.24,130,0.24,"((5, 2, 15, 1, 5, 0, 5), (5, 2, 21, 2, 7, 5, 1...",2876,1.037,1352,1.01
72,AD11,D1,072523,Locomotion,Stim Session,2,Bilateral,"((0, 0, 3, 1, 1, 14, 15), (0, 0, 3, 2, 1, 427,...",50,0.204,...,"((4, 0, 3, 1, 1, 1932, 1933), (4, 1, 6, 2, 2, ...",162,0.242,65,0.29,"((5, 0, 3, 1, 1, 13, 14), (5, 12, 111, 2, 37, ...",1230,2.839,711,2.16
84,P22,D1,062923,Locomotion,Stim Session,1,Bilateral,"((0, 0, 3, 1, 1, 183, 184), (0, 0, 3, 2, 1, 23...",1174,0.207,...,"((4, 1, 9, 1, 3, 141, 144), (4, 1, 9, 2, 3, 14...",1308,0.268,642,0.3,"((5, 1, 12, 1, 4, 0, 4), (5, 4, 33, 2, 11, 4, ...",3593,0.681,1711,0.61
93,P22,D1,072523,Locomotion,Stim Session,2,Bilateral,"((0, 1, 9, 1, 3, 39, 42), (0, 0, 3, 2, 1, 45, ...",859,0.197,...,"((4, 0, 3, 1, 1, 130, 131), (4, 0, 3, 2, 1, 15...",747,0.215,289,0.24,"((5, 3, 30, 1, 10, 0, 10), (5, 2, 21, 2, 7, 11...",3456,0.736,1566,0.74


# Build Out Analysis:

## Compare Pre, Stim, Post of Stimmed Behavior to Pre,Post,Stim of Behaviors

### Step 1

In [198]:
bout_prestimpost={}

count_pre=0
count_stim=0
count_post=0

for col in current_match.columns:
    if "Array" in col:
        bout_col = col.split("Array")[0].strip()
        for row in current_match[col]:
            for tup in row:
                if 0<=tup[5] and tup[6]<= 3000:
                    count_pre+=1
                if tup[5]>=3001 and tup[6]<=21000:
                    count_stim+=1
                if tup[5]>=21001 and tup[6]<=39001:
                    count_post+=1
        bout_prestimpost[bout_col]=(count_pre, count_stim,count_post)

In [226]:
dur_prestimpost={}

count_pre=[]
count_stim=[]
count_post=[]

for col in current_match.columns:
    if "Array" in col:
        dur_col = col.split("Array")[0].strip()
        for row in current_match[col]:
            for tup in row:
                if 0<=tup[5] and tup[6]<= 3000:
                    count_pre.append(tup[4])
                if tup[5]>=3001 and tup[6]<=21000:
                    count_stim.append(tup[4])
                if tup[5]>=21001 and tup[6]<=39001:
                    count_post.append(tup[4])
            pre_dur=sum(count_pre)
            stim_dur=sum(count_stim)
            post_dur=sum(count_post)
        dur_prestimpost[dur_col]=(pre_dur, stim_dur,post_dur)


### Step 2

In [227]:
for key,value in bout_prestimpost.items():
    new_value=tuple(int(x/len(current_match))for x in value)
    bout_prestimpost[key]=new_value

In [228]:
for key,value in dur_prestimpost.items():
    new_value=tuple(int(x/len(current_match))for x in value)
    dur_prestimpost[key]=new_value

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [223]:
print(bout_prestimpost)

{'Right Turn': (70, 309, 243), 'Left Turn': (128, 570, 446), 'Locomotion': (234, 1012, 730), 'Face Groom': (274, 1350, 1126), 'All Other Groom': (297, 1507, 1346), 'All Other': (509, 2688, 2454)}


In [229]:
print(dur_prestimpost)

{'Right Turn': ([99, 215, 308, 419, 574, 722, 756, 1030, 1290, 1457, 1520, 1644, 1729, 1804, 1983, 2013, 2116, 2269, 3328, 3969, 4624, 4824, 5441, 5825, 5906, 6152, 6441, 6486, 6578, 6713, 6752, 6827, 6917, 6931, 7122, 7420, 7428, 7447, 7449, 7468, 7471, 7488, 7489, 7899, 7986, 9600, 11668, 13642, 16189, 18264, 20420, 23230, 25004, 26863], [696, 870, 1283, 2112, 2699, 3585, 3631, 4788, 5727, 6498, 6689, 7037, 7732, 8218, 8962, 8994, 9584, 10219, 13883, 15036, 17232, 18590, 21742, 23221, 23360, 24252, 25331, 26142, 26930, 27129, 27377, 27615, 28530, 30606, 33684, 36699, 36855, 37084, 37117, 37184, 37211, 37526, 37717, 39626, 40318, 52205, 67659, 82460, 97251, 110753, 124375, 139755, 150120, 161719], [773, 840, 1057, 1851, 2190, 2991, 3013, 4006, 4499, 5294, 5370, 5455, 6120, 6449, 7085, 7101, 7714, 8059, 10516, 10867, 11706, 12742, 14876, 16085, 16160, 16814, 17396, 17715, 19961, 21172, 21355, 21741, 22687, 23813, 26100, 29966, 30860, 31244, 31398, 31442, 31840, 32271, 32471, 33661, 344

In [221]:
sum(dur_prestimpost)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [205]:
df_type = type(bout_prestimpost)
print(df_type)

<class 'dict'>


In [206]:
df_type = type(dur_prestimpost)
print(df_type)

<class 'dict'>


### Step 3

In [218]:
# Simulating the condition for selecting a behavior
selected_behavior = current_match['Selected Behavior'].iloc[0]
print(selected_behavior)
selected_genotype = current_match['Genotype'].iloc[0]

# Reorder the rows such that the row matching the condition is the first row
if selected_behavior in boutdf.index:
    boutdf = boutdf.reindex([selected_behavior] + [idx for idx in boutdf.index if idx != selected_behavior])

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
boutdf.plot(kind='bar', ax=ax, width=0.5, color=['#d0e1f9', '#283655', '#1e1f26'])  # Adjust width and colors here

ax.set_ylabel('Counts')
if selected_behavior== 'Right Turn':
    print ('True')
    ax.set_title(f"{selected_genotype} {selected_behavior} {current_match['Fiber Connection'].iloc[0]} Bout Count ({current_match['Session Type'].iloc[0]})")
else:
    ax.set_title(f"{selected_genotype} {selected_behavior} Bout Count ({current_match['Session Type'].iloc[0]})")
ax.set_xticklabels(boutdf.index, rotation=45, ha='right')
plt.legend(title='Period', loc='upper left')
plt.tight_layout()
# Set the current figure as the active figure
plt.gcf()

# Specify the folder path
folder_path = '/Users/kaycei/K on Server (NoMachine)/KayCei/Naturalistic Closed Loop/Overall Data'

# Save the plot as a PNG image with increased DPI
title_str = ax.get_title()
filename = ''.join(e for e in title_str if e.isalnum())  # Remove special characters and spaces
plt.savefig(os.path.join(folder_path, f'{filename}.png'), dpi=300)  # Save as PNG file with increased DPI in the specif

plt.show()

Locomotion


NameError: name 'boutdf' is not defined

In [ ]:
plt.close()

## Stim vs Sham (Repeated ANOVA)

In [159]:
while True:
    finalize_match = input("\nDo you want to finalize matching data? (yes/no): ")
    if finalize_match.lower() == 'yes':
        break  # Exit the loop if the user chooses to finalize matching data
    
    for iteration, (filtering_criteria, columns, current_match) in iteration_list.items():
        print(iteration, filtering_criteria)

    # Prompt the user to select an iteration to go back to
    selected_iteration = input("\nEnter the iteration number to go back to, or 'exit' to exit: ")
    if selected_iteration.lower() == 'exit':
        break  # Exit the loop if the user chooses to exit
    else:
        try:
            selected_iteration = int(selected_iteration)
            if selected_iteration in iteration_list:
                df=iteration_list[selected_iteration][2]
                current_match = iteration_list[selected_iteration][2]
                iteration_list = {k: v for k, v in iteration_list.items() if k <= selected_iteration}
                print('Run cell above as current match')
                break
        except ValueError:
            print("Invalid input. Please enter a valid iteration number or 'exit' to exit.")




Do you want to finalize matching data? (yes/no): no
0 Selected Header: None
1 ['Selected Header: Genotype', 'Filter Value: D1']
2 ['Selected Header: Selected Behavior', 'Filter Value: Right Turn']
5 ['Selected Header: Fiber Connection', 'Filter Value: Contralateral']

Enter the iteration number to go back to, or 'exit' to exit: 1
Run cell above as current match


In [160]:
if finalize_match!='yes':
    # Define a list to store all selections and filtering criteria
    #iteration_list={}
    # Display available columns in groups of 3 with aligned numbering
    num_columns = 3
    columns = df.columns.tolist()
    num_options = len(columns)

    # Calculate the width of each column
    column_width = 32

    # Define the number of spaces for tab spacing
    tab_space = 1

    # Set the number of rows for displaying column options
    num_rows = 13

    while columns:  # Continue filtering until all headers are filtered
        for i in range(num_rows):
            for j in range(i, len(columns), num_rows):
                col_num = j + 1
                col_name = columns[j]
                # Adjust font size for smaller font
                if len(col_name) > column_width:
                    print(f"\033[1;33m{col_num:<2}.\033[1;32m{col_name[:column_width]:<{column_width}}\033[0m", end=" " * tab_space)  # Yellow color for smaller font size
                else:
                    print(f"\033[1;33m{col_num:<2}.{col_name:<{column_width}}\033[0m", end=" " * tab_space)
            print()

        if len(iteration_list)!=1:
            df=current_match
        else:
            pass
        filtering_criteria = []
        # Prompt user to enter the number corresponding to the header
        selected_option = None
        while selected_option is None:
            user_input = input("Enter the number corresponding to the header or '0' to cancel filtering: ")
            try:
                selected_option = int(user_input)
                if selected_option == 0:
                    print("Cell execution stopped.")
                    sys.exit()  # Exit just the current cell
                elif selected_option > len(columns):
                    raise ValueError
            except ValueError:
                print(f"Invalid input. Please enter a number between 1 and {len(columns)} or '0' to cancel filtering.")
                selected_option = None

        print((columns[selected_option-1]))
        selected_header = columns[selected_option - 1]

        # Append the selected header to filtering_criteria
        filtering_criteria.append(f"Selected Header: {selected_header}")

        # Remove the selected header from the list of available columns

        if selected_header in df.columns and all(isinstance(value, str) for value in df[selected_header]):
            avaliable_value=df[selected_header].unique()
            print(avaliable_value)
            filter_value = input(f"Enter the value to filter by {selected_header}: ")
            # Append the filter value to filtering_criteria
            filtering_criteria.append(f"Filter Value: {filter_value}")

        elif 'Array' in selected_header:
            available_elements = {
                0:"Behavior",
                1:"Stim Count",
                2: "Frames", 
                3: "Bout", 
                4: "Duration",
                5: "Start Time",
                6: "End Time",
            }
            print("Available elements:")
            for key, value in available_elements.items():
                print(f"{key}: {value}")

            # Prompt user to select elements
            element_positions_input = input(f"Enter the elements you want to filter by {selected_header} (comma-separated): ")
            print("Selected positions:", selected_positions)  # Add this line for debugging
            if ',' in element_positions_input:
                selected_positions = [int(pos.strip()) for pos in element_positions_input.split(',')]
                # Check if input is valid
                if all(pos in available_elements for pos in selected_positions):
                    combined_filters = []
                    for pos in selected_positions:
                        range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[pos]}?: ")
                        if range_or_exact.lower() == 'exact':
                            filter_value = input(f"Enter the exact value for {available_elements[pos]}: ")
                            # Add condition to the combined filter
                            combined_filters.append(df[selected_header].apply(lambda row: any((x[pos] == filter_value for x in tup)for tup in row)))
                            filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                            filtering_criteria.append(f"Exact Value for {available_elements[pos]}: {filter_value}")
                        elif range_or_exact.lower() == 'range':
                            range_value = input(f"Enter the range for {available_elements[pos]} (min,max): ")
                            min_val, max_val = map(float, range_value.split(','))
                            # Add condition to the combined filter
                            combined_filters.append(df[selected_header].apply(lambda row: any((min_val <= x[pos]<= max_val for x in tup)for tup in row)))
                            filtering_criteria.append(f"Element to filter: {available_elements.get(pos)}")
                            filtering_criteria.append(f"Range for {available_elements[pos]}: {min_val} to {max_val}")
                        else:
                            print("Invalid input. Please enter 'exact' or 'range'.")
                    # Apply combined filter
                    array_df = df[reduce(lambda x, y: x & y, combined_filters)]
                else:
                    print("Value is not an elemental position")
            else:
                selected_positions = int(element_positions_input)
                print(selected_positions)
                if selected_positions in available_elements:
                    range_or_exact = input(f"Do you want an exact value or a range of values for {available_elements[selected_positions]}?: ")
                    if range_or_exact.lower() == 'exact':
                        filter_value= input(f"Enter the exact value for {available_elements[selected_positions]}: ")
                        filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                        filtering_criteria.append(f"Exact Value for {available_elements.get(selected_positions)}: {filter_value}")
                        array_df = df[df[selected_header].apply(lambda row: any((x[selected_positions] == filter_value for x in tup) for tup in row))]
                    elif range_or_exact== 'range':
                        range_value = input(f"Enter the range for {available_elements[selected_positions]} (min,max): ")
                        min_val, max_val = map(float, range_value.split(','))
                        filtering_criteria.append(f"Element to filter: {available_elements.get(selected_positions)}")
                        filtering_criteria.append(f"Range for {available_elements.get(pos)}: {min_val} to {max_val}")
                        array_df=df[df[selected_header].apply(lambda y: any((min_val <= x[selected_positions] <= max_val for x in tup)for tup in y))]
                else:
                    print('An element position was not selected')

        if 'Array'not in selected_header:
            columns.remove(selected_header)

        if len(iteration_list)==1:
            if "Array" not in selected_header:                                          
                current_match=df[df[selected_header] == filter_value]
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)
            elif "Selected Behavior" or 'Fiber Connection' in selected_header:
                current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
                iteration += 1
                iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
            else:
                current_match=array_df
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       
        else:
            previous_match=current_match
            if'Array' in selected_header:
                current_match=array_df
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match) 
            elif "Selected Behavior" or 'Fiber Connection' in selected_header:
                current_match = df[(df[selected_header] == filter_value) | (df[selected_header] == "No Error Value")]
                iteration += 1
                iteration_list[iteration] = (filtering_criteria.copy(), columns.copy(), current_match)
            else:
                current_match=df[df[selected_header]==filter_value]
                iteration+=1
                iteration_list[iteration]=(filtering_criteria.copy(), columns.copy(), current_match)                                       

        # Extracting the headers in the original order from the DataFrame
        original_headers = current_match.columns.tolist()
else:
    pass


1 .Mouse                            14.Total Left Turn Bout Count       27.Face Groom Average Duration in S 
2 .Genotype                         15.Average Duration of Left Turn(se 28.All Other Groom Array            
3 .Date                             16.Total Left Turn Bouts in Stim Bl 29.Total All Other Groom Bout Count 
4 .Selected Behavior                17.Left Turn Average Duration in St 30.Average Duration of All Other Gr 
5 .Session Type                     18.Locomotion Array                 31.Total All Other Groom Bouts in S 
6 .Session Number                   19.Total Locomotion Bout Count      32.All Other Groom Average Duration 
7 .Fiber Connection                 20.Average Duration of Locomotion(s 33.All Other Array                  
8 .Right Turn Array                 21.Total Locomotion Bouts in Stim B 34.Total All Other Bout Count       
9 .Total Right Turn Bout Count      22.Locomotion Average Duration in S 35.Average Duration of All Other(se 
10.Average Duration

SystemExit: 

/Users/kaycei/anaconda3/envs/closed-loop/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [161]:
anovadf=current_match.copy()

In [162]:
count_pre = []
count_stim = []
count_post = []
num_array_columns = sum("Array" in col for col in anovadf.columns)
current_iteration = 0

for col in anovadf.columns:
    if "Array" in col:
        current_iteration += 1
        bout_col_1 = col.split("Array")[0].strip()
        bout_col = '_'.join(bout_col_1.split())
        print(bout_col)
        for row in anovadf[col]:
            pre_count=0
            stim_count=0
            post_count=0
            for tup in row:
                if 0<=tup[5] and tup[6]<= 3000:
                    pre_count+=1
                if tup[5]>=3001 and tup[6]<=21000:
                    stim_count+=1
                if tup[5]>=21001 and tup[6]<=39001:
                    post_count+=1
            count_pre.append(pre_count)
            count_stim.append(stim_count)
            count_post.append(post_count)

        # Add new columns to anovadf before moving to the next array column
        anovadf[f'{bout_col}_Pre_Bout'] = count_pre
        anovadf[f'{bout_col}_Stim_Bout_Count'] = count_stim
        anovadf[f'{bout_col}_Post_Bout'] = count_post

        
        # Clear the count lists if it's not the last iteration
        if current_iteration != num_array_columns:
            count_pre.clear()
            count_stim.clear()
            count_post.clear()

# Add remaining counts to the DataFrame if needed
anovadf[f'{bout_col}_Pre_Bout'] = count_pre
anovadf[f'{bout_col}_Stim_Bout_Count'] = count_stim
anovadf[f'{bout_col}_Post_Bout'] = count_post

Right_Turn
Left_Turn
Locomotion
Face_Groom
All_Other_Groom
All_Other


In [163]:
print(anovadf.columns)

Index(['Mouse', 'Genotype', 'Date', 'Selected Behavior', 'Session Type',
       'Session Number', 'Fiber Connection', 'Right Turn Array',
       'Total Right Turn Bout Count',
       'Average Duration of Right Turn(seconds)',
       'Total Right Turn Bouts in Stim Block',
       'Right Turn Average Duration in Stim Block(seconds)', 'Left Turn Array',
       'Total Left Turn Bout Count', 'Average Duration of Left Turn(seconds)',
       'Total Left Turn Bouts in Stim Block',
       'Left Turn Average Duration in Stim Block(seconds)', 'Locomotion Array',
       'Total Locomotion Bout Count',
       'Average Duration of Locomotion(seconds)',
       'Total Locomotion Bouts in Stim Block',
       'Locomotion Average Duration in Stim Block(seconds)',
       'Face Groom Array', 'Total Face Groom Bout Count',
       'Average Duration of Face Groom(seconds)',
       'Total Face Groom Bouts in Stim Block',
       'Face Groom Average Duration in Stim Block(seconds)',
       'All Other Groom Array'

In [164]:
anovadf.rename(columns={'Session Type': 'Session_Type'}, inplace=True)

In [165]:
anovadf

,Mouse,Genotype,Date,Selected Behavior,Session_Type,Session Number,Fiber Connection,Right Turn Array,Total Right Turn Bout Count,Average Duration of Right Turn(seconds),...,Locomotion_Post_Bout,Face_Groom_Pre_Bout,Face_Groom_Stim_Bout_Count,Face_Groom_Post_Bout,All_Other_Groom_Pre_Bout,All_Other_Groom_Stim_Bout_Count,All_Other_Groom_Post_Bout,All_Other_Pre_Bout,All_Other_Stim_Bout_Count,All_Other_Post_Bout
1,AD12,D1,022024,Locomotion,Stim Session,1,Bilateral,"((1, 0, 3, 1, 1, 13, 14), (1, 1, 9, 2, 3, 19, ...",789,0.199,...,612,22,175,114,6,64,414,253,1517,1635
4,AD14,D1,022624,Locomotion,Stim Session,1,Bilateral,"((1, 1, 12, 1, 4, 46, 50), (1, 0, 3, 2, 1, 62,...",249,0.143,...,95,54,350,809,16,144,188,267,1072,1146
44,AD10,D1,060823,No Error Value,Sham Session,1,No Error Value,"((0, 1, 6, 1, 2, 231, 233), (0, 1, 6, 2, 2, 57...",367,0.184,...,181,6,107,161,4,49,49,165,745,510
46,AD10,D1,062623,Locomotion,Stim Session,1,Bilateral,"((0, 1, 6, 1, 2, 81, 83), (0, 0, 3, 2, 1, 168,...",334,0.216,...,197,29,61,243,2,26,54,220,821,582
48,AD10,D1,062923,No Error Value,Sham Session,1,No Error Value,"((0, 0, 3, 1, 1, 112, 113), (0, 2, 21, 2, 7, 1...",290,0.219,...,223,18,417,177,0,326,122,155,727,587
49,AD10,D1,071023,No Error Value,Sham Session,1,No Error Value,"((0, 2, 18, 1, 6, 20, 26), (0, 1, 12, 2, 4, 58...",212,0.182,...,79,8,665,232,1,457,191,189,1049,318
51,AD10,D1,071323,No Error Value,Sham Session,1,No Error Value,"((0, 0, 3, 1, 1, 75, 76), (0, 0, 3, 2, 1, 546,...",214,0.206,...,10,41,617,687,1,150,94,122,1054,772
53,AD10,D1,071723,No Error Value,Sham Session,1,No Error Value,"((0, 0, 3, 1, 1, 23, 24), (0, 1, 6, 2, 2, 859,...",134,0.193,...,0,85,651,935,24,351,270,153,1059,1102
54,AD10,D1,071923,No Error Value,Sham Session,2,No Error Value,"((0, 0, 3, 1, 1, 261, 262), (0, 1, 6, 2, 2, 26...",218,0.206,...,1,42,275,278,2,171,198,171,965,406
56,AD10,D1,072723,Locomotion,Stim Session,2,Bilateral,"((0, 1, 6, 1, 2, 4, 6), (0, 1, 6, 2, 2, 129, 1...",850,0.204,...,280,18,110,88,8,41,32,151,954,856


In [166]:
len(anovadf[anovadf['Session_Type'] == 'Stim Session'])

9

In [167]:
# Filter columns with '_Stim_Bout_Count' in header
filtered_columns = [col for col in anovadf.columns if '_Stim_Bout_Count' in col]

# Filter columns with 'Bouts in Stim Block' in header
filtered_columns_1 = [col for col in anovadf.columns if 'Bouts in Stim Block' in col]

# List to store column pairs with matching values
matching_columns = []

# Iterate over columns pairwise
for col1 in filtered_columns:
    for col2 in filtered_columns_1:
        # Check if values in columns col1 and col2 match at the same index
        if (anovadf[col1] == anovadf[col2]).all():
            matching_columns.append((col1, col2))

print("Columns with matching values at the same index:")
for pair in matching_columns:
    print(pair)

Columns with matching values at the same index:
('Right_Turn_Stim_Bout_Count', 'Total Right Turn Bouts in Stim Block')
('Left_Turn_Stim_Bout_Count', 'Total Left Turn Bouts in Stim Block')
('Locomotion_Stim_Bout_Count', 'Total Locomotion Bouts in Stim Block')
('Face_Groom_Stim_Bout_Count', 'Total Face Groom Bouts in Stim Block')
('All_Other_Groom_Stim_Bout_Count', 'Total All Other Groom Bouts in Stim Block')
('All_Other_Stim_Bout_Count', 'Total All Other Bouts in Stim Block')


In [168]:
import statsmodels.api as sm
from statsmodels.multivariate.manova import MANOVA

In [170]:
# Define the dependent variables
dependent_vars = ['Right_Turn_Pre_Bout', 'Right_Turn_Stim_Bout_Count',
       'Right_Turn_Post_Bout', 'Left_Turn_Pre_Bout',
       'Left_Turn_Stim_Bout_Count', 'Left_Turn_Post_Bout',
       'Locomotion_Pre_Bout', 'Locomotion_Stim_Bout_Count',
       'Locomotion_Post_Bout', 'Face_Groom_Pre_Bout',
       'Face_Groom_Stim_Bout_Count', 'Face_Groom_Post_Bout',
       'All_Other_Groom_Pre_Bout', 'All_Other_Groom_Stim_Bout_Count',
       'All_Other_Groom_Post_Bout', 'All_Other_Pre_Bout',
       'All_Other_Stim_Bout_Count', 'All_Other_Post_Bout']

# Create a copy of the original dataframe
filtered_data = anovadf.copy()

# Filter out zero values for each dependent variable separately
for var in dependent_vars:
    filtered_data = filtered_data[filtered_data[var] != 0]

# Perform MANOVA on filtered data
manova_model = MANOVA.from_formula(" + ".join(dependent_vars) + " ~ Session_Type", data=filtered_data)
manova_results = manova_model.mv_test()

# Define the title for the Multivariate Linear Model section
stim_session_rows = filtered_data[filtered_data['Session_Type'] == 'Stim Session']
selected_behavior = stim_session_rows['Selected Behavior'].iloc[0]
fiber_connection = stim_session_rows['Fiber Connection'].iloc[0]

title = f"{selected_behavior} {fiber_connection} Multivariate Linear Model (excluding zeros for each dependent variable)"

# Print results with modified title
print(title.center(60, '='))
print(manova_results.summary())


Locomotion Bilateral Multivariate Linear Model (excluding zeros for each dependent variable)
                  Multivariate linear model
                                                              
--------------------------------------------------------------
       Intercept         Value    Num DF Den DF F Value Pr > F
--------------------------------------------------------------
          Wilks' lambda   0.0088 18.0000 7.0000 43.6591 0.0000
         Pillai's trace   0.9912 18.0000 7.0000 43.6591 0.0000
 Hotelling-Lawley trace 112.2663 18.0000 7.0000 43.6591 0.0000
    Roy's greatest root 112.2663 18.0000 7.0000 43.6591 0.0000
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
        Session_Type      Value   Num DF Den DF F Value Pr > F
--------------------------------------------------------------
            Wilks' lambda 0.1464 18.0000 7.0

In [171]:
# Define the dependent variables
dependent_vars = ['Right_Turn_Pre_Bout', 'Right_Turn_Stim_Bout_Count',
       'Right_Turn_Post_Bout', 'Left_Turn_Pre_Bout',
       'Left_Turn_Stim_Bout_Count', 'Left_Turn_Post_Bout',
       'Locomotion_Pre_Bout', 'Locomotion_Stim_Bout_Count',
       'Locomotion_Post_Bout', 'Face_Groom_Pre_Bout',
       'Face_Groom_Stim_Bout_Count', 'Face_Groom_Post_Bout',
       'All_Other_Groom_Pre_Bout', 'All_Other_Groom_Stim_Bout_Count',
       'All_Other_Groom_Post_Bout', 'All_Other_Pre_Bout',
       'All_Other_Stim_Bout_Count', 'All_Other_Post_Bout']

# Extract variables containing the substring "Stim_Bout_Count"
Bout_stim_vars = [var for var in dependent_vars if 'Stim_Bout_Count' in var]

# Create a copy of the original dataframe
filtered_data = anovadf.copy()

# Filter out zero values for each dependent variable separately
for var in Bout_stim_vars:
    filtered_data = filtered_data[filtered_data[var] != 0]

# Perform MANOVA on filtered data
manova_model = MANOVA.from_formula(" + ".join(Bout_stim_vars) + " ~ Session_Type", data=filtered_data)
manova_results = manova_model.mv_test()

# Define the title for the Multivariate Linear Model section
stim_session_rows = filtered_data[filtered_data['Session_Type'] == 'Stim Session']
selected_behavior = stim_session_rows['Selected Behavior'].iloc[0]
fiber_connection = stim_session_rows['Fiber Connection'].iloc[0]

title = f"{selected_behavior} {fiber_connection} Multivariate Linear Model (excluding zeros for each dependent variable)"

# Print results with modified title
print(title.center(60, '='))
print(manova_results.summary())

Locomotion Bilateral Multivariate Linear Model (excluding zeros for each dependent variable)
                  Multivariate linear model
                                                             
-------------------------------------------------------------
       Intercept         Value  Num DF  Den DF F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda  0.0509 6.0000 22.0000 68.4329 0.0000
         Pillai's trace  0.9491 6.0000 22.0000 68.4329 0.0000
 Hotelling-Lawley trace 18.6635 6.0000 22.0000 68.4329 0.0000
    Roy's greatest root 18.6635 6.0000 22.0000 68.4329 0.0000
-------------------------------------------------------------
                                                             
-------------------------------------------------------------
       Session_Type      Value  Num DF  Den DF F Value Pr > F
-------------------------------------------------------------
           Wilks' lambda 0.5375 6.0000 22.0000  3.1552 0.

In [148]:


# Define the file path
file_path = "/Users/kaycei/K on Server (NoMachine)/KayCei/Naturalistic Closed Loop/Totals/Multivariance with manova"

# Save MANOVA results to a file with the title as the filename
filename = os.path.join(file_path, f"{title}.csv")
manova_results_summary = manova_results.summary().as_text()
with open(filename, 'w') as f:
    f.write(manova_results_summary)


In [177]:
anovadf

,Mouse,Genotype,Date,Selected Behavior,Session_Type,Session Number,Fiber Connection,Right Turn Array,Total Right Turn Bout Count,Average Duration of Right Turn(seconds),...,Locomotion_Post_Bout,Face_Groom_Pre_Bout,Face_Groom_Stim_Bout_Count,Face_Groom_Post_Bout,All_Other_Groom_Pre_Bout,All_Other_Groom_Stim_Bout_Count,All_Other_Groom_Post_Bout,All_Other_Pre_Bout,All_Other_Stim_Bout_Count,All_Other_Post_Bout
1,AD12,D1,022024,Locomotion,Stim Session,1,Bilateral,"((1, 0, 3, 1, 1, 13, 14), (1, 1, 9, 2, 3, 19, ...",789,0.199,...,612,22,175,114,6,64,414,253,1517,1635
4,AD14,D1,022624,Locomotion,Stim Session,1,Bilateral,"((1, 1, 12, 1, 4, 46, 50), (1, 0, 3, 2, 1, 62,...",249,0.143,...,95,54,350,809,16,144,188,267,1072,1146
44,AD10,D1,060823,No Error Value,Sham Session,1,No Error Value,"((0, 1, 6, 1, 2, 231, 233), (0, 1, 6, 2, 2, 57...",367,0.184,...,181,6,107,161,4,49,49,165,745,510
46,AD10,D1,062623,Locomotion,Stim Session,1,Bilateral,"((0, 1, 6, 1, 2, 81, 83), (0, 0, 3, 2, 1, 168,...",334,0.216,...,197,29,61,243,2,26,54,220,821,582
48,AD10,D1,062923,No Error Value,Sham Session,1,No Error Value,"((0, 0, 3, 1, 1, 112, 113), (0, 2, 21, 2, 7, 1...",290,0.219,...,223,18,417,177,0,326,122,155,727,587
49,AD10,D1,071023,No Error Value,Sham Session,1,No Error Value,"((0, 2, 18, 1, 6, 20, 26), (0, 1, 12, 2, 4, 58...",212,0.182,...,79,8,665,232,1,457,191,189,1049,318
51,AD10,D1,071323,No Error Value,Sham Session,1,No Error Value,"((0, 0, 3, 1, 1, 75, 76), (0, 0, 3, 2, 1, 546,...",214,0.206,...,10,41,617,687,1,150,94,122,1054,772
53,AD10,D1,071723,No Error Value,Sham Session,1,No Error Value,"((0, 0, 3, 1, 1, 23, 24), (0, 1, 6, 2, 2, 859,...",134,0.193,...,0,85,651,935,24,351,270,153,1059,1102
54,AD10,D1,071923,No Error Value,Sham Session,2,No Error Value,"((0, 0, 3, 1, 1, 261, 262), (0, 1, 6, 2, 2, 26...",218,0.206,...,1,42,275,278,2,171,198,171,965,406
56,AD10,D1,072723,Locomotion,Stim Session,2,Bilateral,"((0, 1, 6, 1, 2, 4, 6), (0, 1, 6, 2, 2, 129, 1...",850,0.204,...,280,18,110,88,8,41,32,151,954,856


In [178]:
import pandas as pd
from scipy.stats import ttest_rel

# Define the group names and corresponding columns from anovadf
group_columns = {
    'Right': ['Right_Turn_Pre_Bout', 'Right_Turn_Stim_Bout_Count', 'Right_Turn_Post_Bout'],
    'Left': ['Left_Turn_Pre_Bout', 'Left_Turn_Stim_Bout_Count', 'Left_Turn_Post_Bout'],
    'Locomotion': ['Locomotion_Pre_Bout', 'Locomotion_Stim_Bout_Count', 'Locomotion_Post_Bout'],
    'Face Groom':['Face_Groom_Pre_Bout','Face_Groom_Stim_Bout_Count', 'Face_Groom_Post_Bout',],
    'All Other Groom':['All_Other_Groom_Pre_Bout', 'All_Other_Groom_Stim_Bout_Count','All_Other_Groom_Post_Bout'],
    'All Other':['All_Other_Pre_Bout','All_Other_Stim_Bout_Count', 'All_Other_Post_Bout'],
}

# Initialize dictionaries to store results
group_comparison_results = {}
pairwise_comparison_results = {}

# Step 1: Subgroup Analysis and Pairwise Comparisons
for group, columns in group_columns.items():
    # Select columns related to the current group from anovadf
    group_data = anovadf[columns]

    # Separate data into "Stim" and "Sham" sessions
    stim_data_1 = group_data[anovadf['Session_Type'] == 'Stim Session']
    sham_data_1 = group_data[anovadf['Session_Type'] == 'Sham Session']

    # Ensure that the number of observations in "Stim" and "Sham" sessions are equal
    min_obs = min(len(stim_data_1), len(sham_data_1))
    if min_obs == 0:
        continue  # Skip this group if one of the sessions has no data
    stim_data = stim_data.sample(min_obs, replace=True, random_state=42)
    sham_data = sham_data.sample(min_obs, replace=True, random_state=42)

    # Perform paired t-test for "Stim" vs. "Sham" within the current group
    t_stat, p_value = ttest_rel(stim_data.values.flatten(), sham_data.values.flatten())
    group_comparison_results[group] = {'t_stat': t_stat, 'p_value': p_value}

    # Perform pairwise comparisons across groups
    other_groups = [other_group for other_group in group_columns.keys() if other_group != group]
    pairwise_comparison_results[group] = {}
    for other_group in other_groups:
        other_group_data = anovadf[group_columns[other_group]]

        # Perform paired t-test between the current group and other groups
        t_stat, p_value = ttest_rel(group_data.values.flatten(), other_group_data.values.flatten())
        pairwise_comparison_results[group][other_group] = {'t_stat': t_stat, 'p_value': p_value}

/Users/kaycei/anaconda3/envs/closed-loop/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/kaycei/anaconda3/envs/closed-loop/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/kaycei/anaconda3/envs/closed-loop/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/Users/kaycei/anaconda3/envs/closed-l

In [179]:
# Print subgroup analysis results
print("Subgroup Analysis Results:")
for group, result in group_comparison_results.items():
    print(f"Group: {group}")
    print(f"T-statistic: {result['t_stat']}")
    print(f"P-value: {result['p_value']}")
    print()

# Print pairwise comparison results
print("Pairwise Comparison Results:")
for group, comparisons in pairwise_comparison_results.items():
    print(f"Group: {group}")
    for other_group, result in comparisons.items():
        print(f"Comparison with {other_group}:")
        print(f"T-statistic: {result['t_stat']}")
        print(f"P-value: {result['p_value']}")
        print()

Subgroup Analysis Results:
Group: Right
T-statistic: inf
P-value: 0.0

Group: Left
T-statistic: inf
P-value: 0.0

Group: Locomotion
T-statistic: inf
P-value: 0.0

Group: Face Groom
T-statistic: inf
P-value: 0.0

Group: All Other Groom
T-statistic: inf
P-value: 0.0

Group: All Other
T-statistic: inf
P-value: 0.0

Pairwise Comparison Results:
Group: Right
Comparison with Left:
T-statistic: 5.8622879867711815
P-value: 8.236173992295729e-08

Comparison with Locomotion:
T-statistic: -0.5115512944523918
P-value: 0.6102760721380696

Comparison with Face Groom:
T-statistic: -2.0615832408861845
P-value: 0.04226675756380987

Comparison with All Other Groom:
T-statistic: 3.644097787616092
P-value: 0.00045829879688565127

Comparison with All Other:
T-statistic: -13.826312453911306
P-value: 1.4369880553404132e-23

Group: Left
Comparison with Right:
T-statistic: -5.8622879867711815
P-value: 8.236173992295729e-08

Comparison with Locomotion:
T-statistic: -3.8574551000900072
P-value: 0.000220743244082

In [182]:
# Define the group names and corresponding columns
group_columns = {
    'Right': ['Right_Turn_Stim_Bout_Count'],
    'Left': ['Left_Turn_Stim_Bout_Count'],
    'Locomotion': ['Locomotion_Stim_Bout_Count'],
    'Face Groom': ['Face_Groom_Stim_Bout_Count'],
    'All Other Groom': ['All_Other_Groom_Stim_Bout_Count'],
    'All Other': ['All_Other_Stim_Bout_Count'],
}

# Initialize dictionaries to store results
group_comparison_results = {}
pairwise_comparison_results = {}

# Step 1: Subgroup Analysis and Pairwise Comparisons
for group, columns in group_columns.items():
    # Select columns related to the current group from anovadf
    group_data = anovadf[columns]

    # Separate data into "Stim" and "Sham" sessions
    stim_data = group_data[anovadf['Session_Type'] == 'Stim Session']
    sham_data = group_data[anovadf['Session_Type'] == 'Sham Session']

    # Ensure that the number of observations in "Stim" and "Sham" sessions are equal
    min_obs = min(len(stim_data), len(sham_data))
    stim_data = stim_data.sample(min_obs, random_state=42)
    sham_data = sham_data.sample(min_obs, random_state=42)

    # Perform paired t-test for "Stim" vs. "Sham" within the current group
    t_stat, p_value = ttest_rel(stim_data.values.flatten(), sham_data.values.flatten())
    group_comparison_results[group] = {'t_stat': t_stat, 'p_value': p_value}

    # Perform pairwise comparisons across groups
    other_groups = [other_group for other_group in group_columns.keys() if other_group != group]
    pairwise_comparison_results[group] = {}
    for other_group in other_groups:
        other_group_data = anovadf[group_columns[other_group]]

        # Perform paired t-test between the current group and other groups
        t_stat, p_value = ttest_rel(group_data.values.flatten(), other_group_data.values.flatten())
        pairwise_comparison_results[group][other_group] = {'t_stat': t_stat, 'p_value': p_value}

In [183]:
# Print subgroup analysis results
print("Subgroup Analysis Results:")
for group, result in group_comparison_results.items():
    print(f"Group: {group}")
    print(f"T-statistic: {result['t_stat']}")
    print(f"P-value: {result['p_value']}")
    print()

# Print pairwise comparison results
print("Pairwise Comparison Results:")
for group, comparisons in pairwise_comparison_results.items():
    print(f"Group: {group}")
    for other_group, result in comparisons.items():
        print(f"Comparison with {other_group}:")
        print(f"T-statistic: {result['t_stat']}")
        print(f"P-value: {result['p_value']}")
        print()

Subgroup Analysis Results:
Group: Right
T-statistic: 0.030677223160743724
P-value: 0.976278481376715

Group: Left
T-statistic: 0.11456610228331175
P-value: 0.9116122703766067

Group: Locomotion
T-statistic: 1.3100802774854972
P-value: 0.22653263800168622

Group: Face Groom
T-statistic: -0.14272747197826777
P-value: 0.8900347138051314

Group: All Other Groom
T-statistic: -0.40761465165676336
P-value: 0.6942465318107864

Group: All Other
T-statistic: 0.5989793311161933
P-value: 0.565757808336383

Pairwise Comparison Results:
Group: Right
Comparison with Left:
T-statistic: 4.356741879465224
P-value: 0.0001604098309101179

Comparison with Locomotion:
T-statistic: -0.5338177398493649
P-value: 0.597680485605151

Comparison with Face Groom:
T-statistic: -1.335696521351235
P-value: 0.19240495865852114

Comparison with All Other Groom:
T-statistic: 2.3749380383827776
P-value: 0.024639203447279476

Comparison with All Other:
T-statistic: -19.08967773514821
P-value: 1.371559970476326e-17

Group: 

In [85]:
len(sham_data_1)

20

In [86]:
len(stim_data_1)

9

In [87]:
stim_data

,All_Other_Pre_Bout,All_Other_Stim_Bout_Count,All_Other_Post_Bout
81,218,1691,1682
3,272,1431,1248
60,233,1002,638
0,263,1371,1484
91,304,1803,1607
20,255,1388,1322
55,135,740,330
45,180,910,913
70,87,462,915


In [88]:
print(min_obs)

9
